In [ ]:
import pandas as pd
import numpy as np
import src.in_out as io
import src.info as info
import src.condition as con
import src.base as base
from itertools import combinations
from src.spread import get_spread
import src.change as change

## Veri Hazırlama

In [ ]:
#Verilere ait path'ler alınıyor
path_list=io.get_path('data')

In [ ]:
#Okunacak sütun adları
cols= ["Time","BID price","ASK price"]
#Tüm veriler okunuyor
master_data=io.read_allData(path_list, cols=cols)
master_data.columns=['time','bid_price','ask_price','name','date']

In [ ]:
#date sütunu Timestamp'e dönüştürülüyor ve tarih ve saat bilgisi birleştiriliyor
master_data['date']=pd.to_datetime(master_data['date'], errors='coerce')
master_data['date'] += pd.to_timedelta(master_data.pop('time').astype(str))

In [ ]:
#zaman dilimi ekleniyor
master_data['time_period']=master_data.date.dt.hour

## Mid Price

In [ ]:
# mid price hesaplanıyor
master_data['mid_price']=base.average_of_series(master_data.bid_price,master_data.ask_price)

In [ ]:
# spread hesaplama için master_data, uygun bir forma dönüştürülüyor
df_mid_price=master_data.pivot(index='date', columns='name', values='mid_price')

## Spread
<img src="img/spread_flowchart.png">
<br/>
Spread, her bir saat için ayrı ayrı hesaplanmıştır

In [ ]:
values={'a_PNLTICK':1,
'a_TICKSIZE':1,
'b_PNLTICK':1,
'b_TICKSIZE':1}

In [ ]:
# pair'lar alınıyor
pairs=combinations(df_mid_price.columns, 2)

In [ ]:
#spread hesaplanıyor
spread_list=[
    base.groupby_date_time(df_mid_price.loc[:,pair]).apply(lambda x: get_spread(x,values))\
        for pair in pairs]
df_spread=pd.concat(spread_list,axis=1).droplevel([0,1])

## Change

In [ ]:
#change alınıyor
df_change=base.groupby_date_time(df_spread).apply(change.get_change)
df_change=df_change.droplevel([0,1])

## Amplitude

In [ ]:
all_pairs_of_amplitude=[]

In [ ]:
for pair_change in df_change:   #Her bir pair alınıyor
    pair_group = list(base.groupby_date_time(df_change[pair_change])) #gün ve saate göre gruplanıyor
    sub_amplitudes=[]
    for i in range(len(pair_group)):
        sub_change=base.to_series(pair_group[i][1])
        sub_amplitudes.append(change.get_amplitude(sub_change)) #pair'a ait her alt grubun amplitudu'u hesaplanıyor
    all_pairs_of_amplitude.append(pd.concat(sub_amplitudes)) # alt amplitude'lar birleştiriliyor

#### 1. Amplitude'u hesaplanan tüm pair'lar all_pairs_of_amplitude adlı dizide tutuluyor.  Hepsi tek DataFrame'de birleştirilmek istenirse aşağıdaki kod çalıştırılabilir.

In [ ]:
from functools import reduce

df_final = reduce(lambda left,right: pd.merge(left,right,on=['date','duration'],how='outer'), all_pairs_of_amplitude)

df_final # NaN'lar birleştirmeden dolayı. ÖNEMSİZ ! 6AU8_6CU8, 2018-09-06 00:00:49 dan başladığından öncesi NaN

# Koşula Göre Amplitude ve Duration Verisinin Yeniden Düzenlenmesi

In [ ]:
#Önce condition modülümüzü import ediyoruz.
import src.condition as condition

In [ ]:
# Hesaplayacağımız verileri saklayabilmek için boş bir dizi yarattık
condition_data = []

In [ ]:
# Python'a ait lambda ifadelerini kullanabilmek için basit bir fonksiyon yazdık
def uygula(data):
    """parametre olarak gelen 'data' daki indeksi kaldırır.
       percentile(.6) ya göre koşul uygulandı.
       percentile silinip yerine medyan() yazılırsa, koşulu medyana göre hesaplar.
       ya da percentile'ın '.6' olan parametresi değiştirilebilir.
    """
    data = data.reset_index(drop=True)
    signs = condition.Sign(data).percentile(.6) #
    return condition.apply(data,signs)

In [ ]:
# bir for döngüsü ile tüm pairlara erişiyoruz
for data in all_pairs_of_amplitude:
    #groupby içerisindeki floor metoduyla veriyi neye göre gruplayacağımızı belirtiyoruz.
    #Burada groupby 2 parametre aldı. ilk parametre ile 'güne göre', 2. parametre ile de 'saate' göre
    #ayırıp, 'uygula' metodunu her gruba ayrı ayrı uygulayacağını belirttik
    """ 'd' -> day
        '2h' -> 2şer saat
        'h'-> birer saat
        'nd' ->n güne böl
        'nh' -> n saate böl gibi        
    """ 
    """.apply metodu ile yapmak istediğimiz işlemi belirtiyoruz
        lambda x : uygula(x) -> her bir grubu sırayla
        x olarak adlandır ve uygula fonksiyonuna parametre olarak gönder
    """
    result = data.groupby([data.date.dt.floor('d'),data.date.dt.floor('h')])\
        .apply(lambda x : uygula(x)).droplevel([0,1])
    #son olarak hesaplanan verileri condition_data adlı listeye atmasını söylüyoruz
    condition_data.append(result)

In [ ]:
# 1. pairi görüntüleyelim

In [ ]:
condition_data[0]

## Aralık Oluşturma

Pair'lerden biri olan **6AU8_6BU8**'deki verileri aralığa yerleştirelim. Bunun için **interval** modülündeki **set_range** ve **set_timeRange** fonksiyonlarını kullanacağız.
<span style="text-decoration:underline">amplitude</span> için **set_range**, <span style="text-decoration:underline">duration</span> için ise **set_timeRange** kullanacağız. Duration'nın veri tipi **timedelta64** olduğundan ayrı bir metot yazıldı.<br/>
Fonksiyonlar **data**, **value**, ve **method** olmak üzere 3 parametre alarak çalışır. Parametreler hakkında kısa bilgi:
1. **data** : aralığa yerleştirilecek veriler. örn. duration veya amplitude
2. **method** : aralık oluşturma yöntemini burada belirtiyoruz. **2** çeşit aralık oluşturma yöntemi var. **'width'** ve **'size'**
   *  **width** : yöntemi seçilirse her bir aralığın genişliği **value** değeri kadardır.
   *  **size** : yöntemi seçilirse **@value** adet aralık oluşturur. Her aralığın genişliği aynıdır.
   *  **value** : yöntem 'width' ise aralık genişliğini, 'size' ise aralık adetini belirtir.

**6AU8_6BU8** pair'ini alalım. Duration'ı **5'er sn'lik aralıklara** yerleştirelim. Amplitude verisini ise **aralık sayısı 100 olacak** ve **her aralığın genişliği eşit** olacak şekilde aralıklara yerleştirelim. Böylece her iki yöntemi de görmüş olacağız

In [ ]:
# 6AU8_6BU8, condition_data adlı dizinin 0. elemanı
ab = condition_data[0]
ab

In [ ]:
#6AU8_6BU8 'den duration ve amplitude sütunları alınıyor
duration = ab['duration']
amplitude = ab['6AU8_6BU8']

In [ ]:
# interval modülünden, kullanacağımız fonksiyonları import ediyoruz
from src.interval import set_range, set_timeRange

In [ ]:
# duration için her biri 5'er sn'lik aralık oluşturuyoruz.
new_duration=set_timeRange(data=duration, value=5, method='width')
new_duration

In [ ]:
# amplitude için her biri eşit büyüklükte 100 adet aralık oluşturuldu ve veriler aralıklara yerleştirildi
new_amplitude = set_range(data=amplitude, value=100, method='size')
new_amplitude

In [ ]:
#pandas'ın concat metodunu kullarak hesapladığımız aralıkları DataFrame'de gösterdik
pd.concat([new_duration,new_amplitude],axis=1)

Yukarıda görüldüğü gibi verilerimizi aralığa yerleştirdik. Fakat her biri Interval türünden olduğundan dolayı hareket alanımızı kısıtlayacaktır.
Örneğin aşağıdaki gibi sütun verileriyle bir toplama işlemi gerçekleştirmek istediğimizi varsayalım :
```python
new_amplitude.sum()
```
Bu hamlemiz aşağıdaki hata ile sonuçlanacktır:
```
TypeError: Categorical cannot perform the operation sum
```
Aralık tanımlamadaki amaç, bu verileri **kategorilendirmek** olduğundan, her bir aralığın yalnızca **sağ değeri** ya da **ortalamasıyla** çalışmamız işlemlerimizin sonucunu etkilemeyecektir. Bize istatistiksel hesaplamalar veya grafik işlemlerinde kolaylık sağlayacaktır.<br/>

Şimdi duration ve amplitude aralıklarının ortalamasını aşağıdaki gibi alalım. Önce **interval** modülümüzden **get_mid** fonksiyonumuzu import edelim.

In [ ]:
from src.interval import get_mid

In [ ]:
#Amplitude için aralıkların ortalamasını al ve veriyi float'a dönüştür
new_amplitude=get_mid(new_amplitude).astype('float64')

In [ ]:
#Duration için aralıkların ortalamasını al ve veriyi timedelta64'e dönüştür
new_duration=get_mid(new_duration).astype('timedelta64')

In [ ]:
#Verileri  DataFrame ile gösterelim ve 'new_ab' adlı değişkende saklayalım
new_ab=pd.concat([new_duration,new_amplitude],axis=1)
new_ab

Aralık işlemleri bu kadar. Şimdi oluşturduğumuz **new_ab** ile **Density İşlemleri**'ni gerçekleştireceğiz.

## Density

In [ ]:
%load_ext autoreload

In [ ]:
%autoreload

### 1. Joint Density
Bu bölümde aşağıdaki işlemler yapılmaktadır:
1. **get_frequency:** Her bir  {duration,amplitude} değerinden kaç tane olduğunu bulma işlemidir. Örneğin duration'ı 5 amplitude'u 10 olan 133 veri var, duration:5 amplitude:7 olan 23 veri var. **Parametre olarak** yalnızca **duration** ve **amplitude** içeren bir **DataFrame** alır. 
2. **joint_density:** Her bir frekansın, toplam frekansa bölünerek tabloda ilgili yere yerleştirilmiş halidir.Daha sonra bu tablonun yatay ve dikey toplamlarını alarak **marginal density** elde edilecektir. **Parametre olarak** yalnızca **duration** ve **amplitude** içeren bir **DataFrame** alır. 

**density** modülünden kullanacağımız fonksiyonları import edelim ve işlemleri gerçekleştirelim

In [ ]:
from src.density import get_frequency, joint_density

In [ ]:
# frekans hesaplanıyor
get_frequency(new_ab)

In [ ]:
#joint density hesaplanıyor ve joint adlı bir değişkende saklanıyor
joint=joint_density(new_ab)
joint

Şimdi bulduğumuz **joint density**'nin **yataydaki ve dikeydeki toplamını** alalım. Fakat bulacağımız toplamları, aşağıdaki **Marjinal Density** tablosundaki gibi, tekrardan joint dataframe'ine aktarmayacağız. **vertical_total** ve **horizontal_total** adlı iki değişkende tutacağız. Daha sonra ise bunları kullanarak **Conditional Density** hesaplayacağız.
```
Örnek Tablo
| xy           | 4             | 6             | 8             | yatay toplam |
|--------------|---------------|---------------|---------------|--------------|
| 1            | 0,07547169811 | 0,2830188679  | 0,1320754717  | 0,4905660377 |
| 2            | 0,05031446541 | 0,07547169811 | 0,1446540881  | 0,2704402516 |
| 3            | 0,1257861635  | 0,04402515723 | 0,06918238994 | 0,2389937107 |
| dikey toplam | 0,251572327   | 0,4025157233  | 0,3459119497  | 1            |
```
Ayrıca aşağıda bulacağımız **vertical_total** ve **horizontal_total**'in toplamı **1'e eşittir**.

In [ ]:
# Dikey toplam
vertical_total = joint.agg('sum') #agg metodu pandas'a ait
vertical_total

In [ ]:
#Yatay toplam
horizontal_total = joint.agg('sum',axis=1)
horizontal_total

### 2. Conditional Density
Conditional Density 2 şekilde hesaplanır. x eksenine veya y eksenine göre(x:duration, y:amplitude temsil eder).<br/>
1. **Conditional Density / y** için: joint density'deki(**joint**) her değer, karşılık gelen dikey toplama(**vertical_total**) bölünerek hesaplanır.
2. **Conditional Density / x** için: joint density'deki(**joint**) her değer, karşılık gelen yatay toplama(**horizontal_total**) bölünerek hesaplanır.
```
Örnek Tablo (Conditional Density / y)
| xy | 4   | 6        | 8        |
|----|-----|----------|----------|
| 1  | 0,3 | 0,703125 | 0,381... |
| 2  | 0,2 | 0,1875   | 0,418..  |
| 3  | 0,5 | 0,109375 | 0,2      |
|    | 1   | 1        | 1        |
```

In [ ]:
# Conditional Density / y
(joint/vertical_total)

In [ ]:
# Conditional Density / x
(joint.T/horizontal_total)